In [1]:
## for data
import json
import pandas as pd
import numpy as np
from sklearn import metrics, manifold
## for processing
import re
import nltk
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for w2v
import gensim
import gensim.downloader as gensim_api
## for bert
import transformers
import pandas as pd
import tarfile
import os
import gc
import random

import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score


import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Dense, Input, Dropout, GlobalAveragePooling1D,Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, Callback, ReduceLROnPlateau

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows',5000)
from tqdm.autonotebook import tqdm

import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:39: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [23]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.svm import SVC


data = pd.read_csv("/kaggle/input/red-vf/RED_VF.csv", encoding='utf-8')
pd.set_option('display.max_colwidth', None)
data=data.drop(['Rating'] , axis=1)
data.head(5)

,Review
0,"Habitant à une trentaine de kilomètres de la frontière suisse j’ai eu le droit à un hors forfait de plus de 80€.\nEt Sfr ne m’a jamais prévenu que j’étais sur le réseau Suisse et pour causse je n’avais jamais eu de soucis précédemment !\nÇa fait mal de se faire voler comme cela, c’est de l’extorsion de fond!"
1,"Horrible.\n\n\n4g qui bug 24/24h, service client inexistant (ce sont des freelance qui ne parlent même pas bien le français et qui ne vous aideront à rien donc vous êtes seul derrière votre écran si vous avez un problème)\n\narnaque."
2,"J’ai souscris à une offre sport et résilier cette dernière avant la fin du premier mois d’inscription (moins de 30 jours).\nMais ils m’ont facturés quand même le deuxième mois sous prétexte que il faut résilier l’abonnement avant la fin du mois calendaire. Une fois basculé sur un nouveau mois calendaire, ils ont pour obligation de facturer même si on s’est désabonné avant que cela fasse 30 jours … C’est la première fois que je vois un tel fonctionnement. Mais c’est pas tout, j’ai même pas accès au service 😂\nDonc ils me prennent de l’argent mais j’ai pas le service. C’est pas beau ça ?\nMerci REDBYSFR."
3,"Minable! Et encore je suis gentil. Impossible de recevoir et d'envoyer un SMS. Impossible de joindre un conseillé car il faut passer par des informations en ligne, informations qui ne servent absolument à rien. N'espérez pas trouver de l'aide auprès des conseillers SFR dans une boutique car ils vous envoient sur les roses car vous n'êtes pas client chez-eux. Je viens de prendre un forfait chez RED by SFR il y a 8 jours... Je retourne chez mon ancien fournisseur. Au moins je vais avoir un forfait qui fonctionne et la possibilité d'avoir un vrai conseillé en ligne en cas de problème."
4,"Au mois de février, un démarcheur, M. Sadou MANE nous as contacté afin que l'on passe notre Box internet + forfaits mobile chez Red by SFR.\nIl nous a proposé une BOX internet et vendu 2 forfaits mobiles à 10€ chaque.\nSurprise, nous avons reçus en février et mars des factures de 16.99€ pour chaun de nos forfaits mobile au lieu de 10€ convenu par téléphone.\nJ'ai passé mon temps à contacter les services clients, fidélisation, résiliation verdict : on ne peux rien faire cette offre n'existe pas. C'est un scandale ! On nous vend un produit qui n'existe pas et on ne peux plus rien faire ? C'est une vraie arnaque. J'ai demandé à faire réécouter ma conversation téléphonique, mais personne ne prends en compte ma demande. Impossible de joindre un responsable. Un scandale !"


In [197]:
import re
import string
import emoji
import unicodedata
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer

nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text, stem=False):

    # Remove accents from text
    text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode("utf-8")

    # Remove emojis from text
#     text = emoji.get_emoji_regexp().sub(u'', text)

    # Remove URLs and email addresses from text
    text = re.sub(r'\b(?:https?://|www\.)\S+\b', '', text)
    text = re.sub(r'\b[\w.]+?@\w+?\.\w{2,4}\b', '', text)

    # Remove special characters from text
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # Remove punctuation from text
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize text into words
    words = nltk.word_tokenize(text)

    # Remove stopwords from text
    stop_words = set(stopwords.words('french'))
    words = [word for word in words if word not in stop_words]

    # Apply stemming to text
    if stem:
        stemmer = FrenchStemmer()
        words = [stemmer.stem(word) for word in words]

    # Join words back into a string
    text = ' '.join(words)

    return text

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [24]:
data['Review'] = data['Review'].apply(preprocess_text)

NameError: name 'preprocess_text' is not defined

In [25]:
data.shape

(3452, 1)

In [38]:
labels = {"Résiliation": ["fuir","resilié","résiliée","résiliation","resiliation"],
          "ServiceClient": ["après vente","service après vente","rappeler","conseiller","service clients","service client", "assistance", "hotline", "support", "aide", "conseiller", "conseil","SAV"],
          "forfait/Offre": ["forfait","promo","promotion", "offre", "rabais", "réduction", "avantage", "bon plan", "code promo"],
          "Facturation": ["surfacturation","facturation","facture", "paiement", "montant", "échéance", "remboursement", "prélèvement", "tva"],
          "Débit/Internet": ["internet","vitesse", "débit", "connexion", "lenteur", "ralentissement", "instabilité", "buffering"],
          "Installation/ServiceTech": ["tech","Service Technique","Technicien","installation", "paramétrage", "configuration", "réglage", "installation", "paramètre", "logiciel"],
          "Contrat": ["Contrat","engagement", "durée", "renouvellement", "engager", "souscrire", "clause"],
          "Fibre": ["fibre","fibre optique"],
          "Couverture/Réseau": ["coupure","couverture","réseau", "câble", "antenne", "télévision", "wifi", "téléphone", "signal"]}

Lorsque la troisième catégorie a le même nombre de votes que la deuxième, cela signifie que les mots-clés associés à ces deux catégories ont été trouvés à peu près autant de fois dans le texte. Dans ce cas, il n'est pas possible de déterminer avec certitude laquelle des deux catégories est la plus pertinente pour le texte, car elles ont toutes les deux une importance similaire.

En retournant les deux premières catégories dans ce cas, la fonction choisit de se concentrer sur les deux catégories qui ont clairement reçu plus de votes que la troisième, ce qui suggère qu'elles sont plus fortement associées au texte. De cette façon, la fonction peut aider à fournir des informations utiles aux utilisateurs en limitant le nombre de catégories proposées et en se concentrant sur celles qui ont le plus de pertinence pour le texte.

La fonction compare le text avec une liste de keywords préalablement définis pour chaque catégorie, qui sont stockés dans un dictionnaire nommé labels. Si un keyword correspond à un mot dans le text, la catégorie correspondante reçoit un vote.

La fonction retourne les trois catégories ayant reçu le plus grand nombre de votes, sans compter les doublons. Si la troisième catégorie a reçu le même nombre de votes que la deuxième, seules deux catégories sont renvoyées. Les catégories sont ensuite concaténées dans une chaîne de caractères avec un saut de ligne entre chaque catégorie

In [39]:
def get_categories(text):
    category_votes = {}
    for category, keywords in labels.items():
        for keyword in keywords:
            if keyword in text.lower():
                if category in category_votes:
                    category_votes[category] += 1
                else:
                    category_votes[category] = 1
    categories = sorted(category_votes, key=category_votes.get, reverse=True)[:3]
    # Élimination des doublons
    categories = list(set(categories))
    # Retourne deux catégories si la troisième a le même nombre de votes que la deuxième
    if len(categories) == 3 and category_votes[categories[2]] == category_votes[categories[1]]:
        categories = categories[:2]
    # Joindre les catégories 
    category_column = " ".join(categories)
    return category_column

data["categories"] = data["Review"].apply(get_categories)

In [40]:
data.head(80)

,Review,categories
0,Habitant à une trentaine de kilomètres de la frontière suisse j’ai eu le droit à un hors forfait de plus de 80€. Et Sfr ne m’a jamais prévenu que j’étais sur le réseau Suisse et pour causse je n’avais jamais eu de soucis précédemment ! Ça fait mal de se faire voler comme cela c’est de l’extorsion de fond!,forfait/Offre Couverture/Réseau
1,Horrible. 4g qui bug 24/24h service client inexistant (ce sont des freelance qui ne parlent même pas bien le français et qui ne vous aideront à rien donc vous êtes seul derrière votre écran si vous avez un problème) arnaque.,ServiceClient
2,J’ai souscris à une offre sport et résilier cette dernière avant la fin du premier mois d’inscription (moins de 30 jours). Mais ils m’ont facturés quand même le deuxième mois sous prétexte que il faut résilier l’abonnement avant la fin du mois calendaire. Une fois basculé sur un nouveau mois calendaire ils ont pour obligation de facturer même si on s’est désabonné avant que cela fasse 30 jours … C’est la première fois que je vois un tel fonctionnement. Mais c’est pas tout j’ai même pas accès au service 😂 Donc ils me prennent de l’argent mais j’ai pas le service. C’est pas beau ça ? Merci REDBYSFR.,forfait/Offre Facturation
3,Minable! Et encore je suis gentil. Impossible de recevoir et d'envoyer un SMS. Impossible de joindre un conseillé car il faut passer par des informations en ligne informations qui ne servent absolument à rien. N'espérez pas trouver de l'aide auprès des conseillers SFR dans une boutique car ils vous envoient sur les roses car vous n'êtes pas client chez-eux. Je viens de prendre un forfait chez RED by SFR il y a 8 jours... Je retourne chez mon ancien fournisseur. Au moins je vais avoir un forfait qui fonctionne et la possibilité d'avoir un vrai conseillé en ligne en cas de problème.,forfait/Offre ServiceClient
4,Au mois de février un démarcheur M. Sadou MANE nous as contacté afin que l'on passe notre Box internet + forfaits mobile chez Red by SFR. Il nous a proposé une BOX internet et vendu 2 forfaits mobiles à 10€ chaque. Surprise nous avons reçus en février et mars des factures de 16.99€ pour chaun de nos forfaits mobile au lieu de 10€ convenu par téléphone. J'ai passé mon temps à contacter les services clients fidélisation résiliation verdict : on ne peux rien faire cette offre n'existe pas. C'est un scandale ! On nous vend un produit qui n'existe pas et on ne peux plus rien faire ? C'est une vraie arnaque. J'ai demandé à faire réécouter ma conversation téléphonique mais personne ne prends en compte ma demande. Impossible de joindre un responsable. Un scandale !,forfait/Offre Facturation
5,Je prends un abonnement + achat d'un téléphone pour SFR RED 200 GO; la ligne est opérationnelle le 18/03/23 après réception du matériel et carte Sim. Tout fonctionne bien. Le 20/03/23 plus de communication ni sortante ni entrante. SFR a coupé le ligne arbitrairement sans prévenir pour motif de problème lié a la sureté et le réclame des documents déjà fourni(RIB Justificatif domicile mail de confirmation sfr ouverture de ligne et pièce identité). Ce qui est inadmissible: couper une ligne sans prévenir ni demander des éventuels documents manquants. Cette coupure de ligne met en danger des personnes vivants ds une région isolée. C est un procédé de voyous.,Couverture/Réseau
6,RED by SFR. Je suis client de cette société SFR depuis longtemps : RED by SFR mais depuis quelques mois rien ne va plus le service client est bien en dessous de ce qu’il devrait être. Le dernier appel que j’ai fait s’est mal termine et ma demande n’a pas été réglée. On a au bout du fil des personnes bien souvent incompétentes ou qu’ils n’écoutent pas ce qu’on leur dit. Mes problèmes n’ont jamais été réglés et j’ai même fortement pensé à changer de fournisseur. De plus depuis quelques mois la rapidité des connections internet laisse à désirer et ils veulent nous vendre la fibre ! Je suis éligible a la fibre mais quand je vois comment cela a été installé (instal

In [41]:
data['Review'] = data['Review'].str.replace('\n', ' ')
data['Review'] = data['Review'].str.replace(',', ' ')

In [44]:
data.head(50)

,Review,categories
0,Habitant à une trentaine de kilomètres de la frontière suisse j’ai eu le droit à un hors forfait de plus de 80€. Et Sfr ne m’a jamais prévenu que j’étais sur le réseau Suisse et pour causse je n’avais jamais eu de soucis précédemment ! Ça fait mal de se faire voler comme cela c’est de l’extorsion de fond!,forfait/Offre Couverture/Réseau
1,Horrible. 4g qui bug 24/24h service client inexistant (ce sont des freelance qui ne parlent même pas bien le français et qui ne vous aideront à rien donc vous êtes seul derrière votre écran si vous avez un problème) arnaque.,ServiceClient
2,J’ai souscris à une offre sport et résilier cette dernière avant la fin du premier mois d’inscription (moins de 30 jours). Mais ils m’ont facturés quand même le deuxième mois sous prétexte que il faut résilier l’abonnement avant la fin du mois calendaire. Une fois basculé sur un nouveau mois calendaire ils ont pour obligation de facturer même si on s’est désabonné avant que cela fasse 30 jours … C’est la première fois que je vois un tel fonctionnement. Mais c’est pas tout j’ai même pas accès au service 😂 Donc ils me prennent de l’argent mais j’ai pas le service. C’est pas beau ça ? Merci REDBYSFR.,forfait/Offre Facturation
3,Minable! Et encore je suis gentil. Impossible de recevoir et d'envoyer un SMS. Impossible de joindre un conseillé car il faut passer par des informations en ligne informations qui ne servent absolument à rien. N'espérez pas trouver de l'aide auprès des conseillers SFR dans une boutique car ils vous envoient sur les roses car vous n'êtes pas client chez-eux. Je viens de prendre un forfait chez RED by SFR il y a 8 jours... Je retourne chez mon ancien fournisseur. Au moins je vais avoir un forfait qui fonctionne et la possibilité d'avoir un vrai conseillé en ligne en cas de problème.,forfait/Offre ServiceClient
4,Au mois de février un démarcheur M. Sadou MANE nous as contacté afin que l'on passe notre Box internet + forfaits mobile chez Red by SFR. Il nous a proposé une BOX internet et vendu 2 forfaits mobiles à 10€ chaque. Surprise nous avons reçus en février et mars des factures de 16.99€ pour chaun de nos forfaits mobile au lieu de 10€ convenu par téléphone. J'ai passé mon temps à contacter les services clients fidélisation résiliation verdict : on ne peux rien faire cette offre n'existe pas. C'est un scandale ! On nous vend un produit qui n'existe pas et on ne peux plus rien faire ? C'est une vraie arnaque. J'ai demandé à faire réécouter ma conversation téléphonique mais personne ne prends en compte ma demande. Impossible de joindre un responsable. Un scandale !,forfait/Offre Facturation
5,Je prends un abonnement + achat d'un téléphone pour SFR RED 200 GO; la ligne est opérationnelle le 18/03/23 après réception du matériel et carte Sim. Tout fonctionne bien. Le 20/03/23 plus de communication ni sortante ni entrante. SFR a coupé le ligne arbitrairement sans prévenir pour motif de problème lié a la sureté et le réclame des documents déjà fourni(RIB Justificatif domicile mail de confirmation sfr ouverture de ligne et pièce identité). Ce qui est inadmissible: couper une ligne sans prévenir ni demander des éventuels documents manquants. Cette coupure de ligne met en danger des personnes vivants ds une région isolée. C est un procédé de voyous.,Couverture/Réseau
6,RED by SFR. Je suis client de cette société SFR depuis longtemps : RED by SFR mais depuis quelques mois rien ne va plus le service client est bien en dessous de ce qu’il devrait être. Le dernier appel que j’ai fait s’est mal termine et ma demande n’a pas été réglée. On a au bout du fil des personnes bien souvent incompétentes ou qu’ils n’écoutent pas ce qu’on leur dit. Mes problèmes n’ont jamais été réglés et j’ai même fortement pensé à changer de fournisseur. De plus depuis quelques mois la rapidité des connections internet laisse à désirer et ils veulent nous vendre la fibre ! Je suis éligible a la fibre mais quand je vois comment cela a été installé (instal

In [43]:
data.to_csv('output_BASE.csv')

In [172]:
# data['categories'] = data['categories'].fillna('')

In [173]:
# print(type(data['categories']))

In [45]:
data['Facturation'] = [1 if 'Facturation' in i else 0 for i in data['categories']]
data['Service client'] = [1 if 'Service client' in i else 0 for i in data['categories']]
data['Installation/Service Tech'] = [1 if 'Installation/Service Tech' in i else 0 for i in data['categories']]
data['Vitesse Internet'] = [1 if 'Vitesse Internet' in i else 0 for i in data['categories']]
data['Résiliation'] = [1 if 'Résiliation' in i else 0 for i in data['categories']]
data['forfait/Offre'] = [1 if 'forfait/Offre' in i else 0 for i in data['categories']]
data['Contrat'] = [1 if 'Contrat' in i else 0 for i in data['categories']]
data['Fibre'] = [1 if 'Fibre' in i else 0 for i in data['categories']]
data['Couverture Réseau'] = [1 if 'Couverture Réseau' in i else 0 for i in data['categories']]


In [46]:
data.head()

,Review,categories,Facturation,Service client,Installation/Service Tech,Vitesse Internet,Résiliation,forfait/Offre,Contrat,Fibre,Couverture Réseau
0,Habitant à une trentaine de kilomètres de la frontière suisse j’ai eu le droit à un hors forfait de plus de 80€. Et Sfr ne m’a jamais prévenu que j’étais sur le réseau Suisse et pour causse je n’avais jamais eu de soucis précédemment ! Ça fait mal de se faire voler comme cela c’est de l’extorsion de fond!,forfait/Offre Couverture/Réseau,0,0,0,0,0,1,0,0,0
1,Horrible. 4g qui bug 24/24h service client inexistant (ce sont des freelance qui ne parlent même pas bien le français et qui ne vous aideront à rien donc vous êtes seul derrière votre écran si vous avez un problème) arnaque.,ServiceClient,0,0,0,0,0,0,0,0,0
2,J’ai souscris à une offre sport et résilier cette dernière avant la fin du premier mois d’inscription (moins de 30 jours). Mais ils m’ont facturés quand même le deuxième mois sous prétexte que il faut résilier l’abonnement avant la fin du mois calendaire. Une fois basculé sur un nouveau mois calendaire ils ont pour obligation de facturer même si on s’est désabonné avant que cela fasse 30 jours … C’est la première fois que je vois un tel fonctionnement. Mais c’est pas tout j’ai même pas accès au service 😂 Donc ils me prennent de l’argent mais j’ai pas le service. C’est pas beau ça ? Merci REDBYSFR.,forfait/Offre Facturation,1,0,0,0,0,1,0,0,0
3,Minable! Et encore je suis gentil. Impossible de recevoir et d'envoyer un SMS. Impossible de joindre un conseillé car il faut passer par des informations en ligne informations qui ne servent absolument à rien. N'espérez pas trouver de l'aide auprès des conseillers SFR dans une boutique car ils vous envoient sur les roses car vous n'êtes pas client chez-eux. Je viens de prendre un forfait chez RED by SFR il y a 8 jours... Je retourne chez mon ancien fournisseur. Au moins je vais avoir un forfait qui fonctionne et la possibilité d'avoir un vrai conseillé en ligne en cas de problème.,forfait/Offre ServiceClient,0,0,0,0,0,1,0,0,0
4,Au mois de février un démarcheur M. Sadou MANE nous as contacté afin que l'on passe notre Box internet + forfaits mobile chez Red by SFR. Il nous a proposé une BOX internet et vendu 2 forfaits mobiles à 10€ chaque. Surprise nous avons reçus en février et mars des factures de 16.99€ pour chaun de nos forfaits mobile au lieu de 10€ convenu par téléphone. J'ai passé mon temps à contacter les services clients fidélisation résiliation verdict : on ne peux rien faire cette offre n'existe pas. C'est un scandale ! On nous vend un produit qui n'existe pas et on ne peux plus rien faire ? C'est une vraie arnaque. J'ai demandé à faire réécouter ma conversation téléphonique mais personne ne prends en compte ma demande. Impossible de joindre un responsable. Un scandale !,forfait/Offre Facturation,1,0,0,0,0,1,0,0,0


In [176]:
data.dtypes

Review                       object
categories                   object
Facturation                   int64
Service client                int64
Installation/Service Tech     int64
Vitesse Internet              int64
Résiliation                   int64
forfait/Offre                 int64
Contrat                       int64
Fibre                         int64
Couverture Réseau             int64
dtype: object

In [177]:
pip install sacremoses

Note: you may need to restart the kernel to use updated packages.


In [ ]:
warnings.filterwarnings('ignore')
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import TFAutoModel, AutoTokenizer
import transformers as trf


tokenizer = AutoTokenizer.from_pretrained("flaubert/flaubert_large_cased")
bert = trf.TFBertModel.from_pretrained("flaubert/flaubert_large_cased",from_pt=True)

In [179]:
warnings.filterwarnings('ignore')

lenx = []
for txt in data['Review']:
  encodes = tokenizer.encode_plus(txt) #we have already defined the tokenizer
  lenx.append(len(encodes['input_ids']))

Token indices sequence length is longer than the specified maximum sequence length for this model (802 > 512). Running this sequence through the model will result in indexing errors


In [180]:
print('Max len: {}'.format(np.max(lenx)))
MAX_LEN = np.max(lenx)+10

Max len: 2205


In [181]:
def encode(texts, MAX_LEN):
  input_ids =[]
  attention_masks =[]
  token_type_ids =[]

  for text in tqdm(texts):
    tokens = tokenizer.encode_plus(text, max_length= MAX_LEN, padding='max_length', return_tensors='tf',truncation=True)
    input_ids.append(tokens['input_ids'][0])
    attention_masks.append(tokens['attention_mask'][0])
    token_type_ids.append(tokens['token_type_ids'][0])
    # print(tokens['input_ids'][0])


  return np.array(input_ids), np.array(attention_masks), np.array(token_type_ids)

In [182]:
input_ids_train, attention_mask_train, token_type_ids_train =encode(data['Review'],MAX_LEN)
input_ids_train.shape

  0%|          | 0/3452 [00:00<?, ?it/s]

(3452, 2215)

In [183]:
y_fact = np.array([1 if 'Facturation' in i else 0 for i in data['categories']])
y_resilience = np.array([1 if 'Résiliation' in i else 0 for i in data['categories']])
y_client = np.array([1 if 'Service client' in i else 0 for i in data['categories']])
y_tech = np.array([1 if 'Installation/Service Tech' in i else 0 for i in data['categories']])
y_promo = np.array([1 if 'Promotion/Offre' in i else 0 for i in data['categories']])
y_internet = np.array([1 if 'Vitesse Internet' in i else 0 for i in data['categories']])
y_contrat = np.array([1 if 'Contrat' in i else 0 for i in data['categories']])
y_fibre = np.array([1 if 'Fibre' in i else 0 for i in data['categories']])
y_couverture = np.array([1 if 'Couverture Réseau' in i else 0 for i in data['categories']])

In [184]:
#Hyperparameters
batch_size=64
epochs=10
lr=0.001
SEED = 5520

In [185]:
# Model Building
def build_model():
    input_id = Input(shape=(MAX_LEN,), dtype=tf.int32)
    att_mask = Input(shape=(MAX_LEN,), dtype=tf.int32)
    token_type = Input(shape=(MAX_LEN,), dtype=tf.int32)

    embed = bert([input_id, att_mask, token_type])

    cls_output = embed[0][:, 0, :]  # embedding of the [CLS] token

    facture_out = Dropout(0.1)(cls_output)
    facture_out = Dense(1, activation='sigmoid')(facture_out)

    resil_out = Dropout(0.1)(cls_output)
    resil_out = Dense(1, activation='sigmoid')(resil_out)

    client_out = Dropout(0.1)(cls_output)
    client_out = Dense(1, activation='sigmoid')(client_out)

    tech_out = Dropout(0.1)(cls_output)
    tech_out = Dense(1, activation='sigmoid')(tech_out)

    promo_out = Dropout(0.1)(cls_output)
    promo_out = Dense(1, activation='sigmoid')(promo_out)

    internet_out = Dropout(0.1)(cls_output)
    internet_out = Dense(1, activation='sigmoid')(internet_out)

    contrat_out = Dropout(0.1)(cls_output)
    contrat_out = Dense(1, activation='sigmoid')(contrat_out)

    fibre_out = Dropout(0.1)(cls_output)
    fibre_out = Dense(1, activation='sigmoid')(fibre_out)

    couverture_out = Dropout(0.1)(cls_output)
    couverture_out = Dense(1, activation='sigmoid')(couverture_out)

    model = Model(inputs=[input_id, att_mask, token_type], outputs=[facture_out, resil_out, client_out, tech_out, promo_out, internet_out, contrat_out, fibre_out, couverture_out])
    optimizer = Adam(learning_rate=lr)

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model


In [186]:
#model building
bert_model = build_model()

#model training

bert_model.fit([input_ids_train, attention_mask_train, token_type_ids_train], 
               [y_fact, y_resilience, y_client, y_tech, y_promo, y_internet, y_contrat, y_fibre, y_couverture],
               batch_size=batch_size, 
               epochs=epochs,
               validation_split=0.2)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'model_4/tf_bert_model_2/bert/embeddings/Gather_1' defined at (most recent call last):
    File "/opt/conda/lib/python3.7/runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "/opt/conda/lib/python3.7/runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/conda/lib/python3.7/site-packages/traitlets/config/application.py", line 1041, in launch_instance
      app.start()
    File "/opt/conda/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/opt/conda/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/opt/conda/lib/python3.7/asyncio/base_events.py", line 541, in run_forever
      self._run_once()
    File "/opt/conda/lib